# Model Prediction
This notebook implements prediction using sliding window approach.

## Imports and Setup

In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

## File Location

In [ ]:
version = '20200502'
data_dir = "../data/"
output_dir = "../outputs/"
input_file = data_dir + '{}_dataset.csv'.format(version)

images_dir = data_dir + 'images/'
indices_dir = data_dir + 'indices/'
pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
tmp_dir = data_dir + 'tmp/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

areas = ['maicao']

## Load Dataset

In [ ]:
data = pd.read_csv(input_file).reset_index(drop=True)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

## Resample Dataset

In [ ]:
data = model_utils.resample(data, num_neg_samples=30000, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

## Machine Learning Pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
SEED = 42

## Define Features and Target Label

In [ ]:
label = 'target'
features = [column  for column in data.columns[:-2]]
data[label] = data[label].replace({2:0, 3:0})

splits = data[['area']]
X = data[features]
y = data[label]

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

## Define Best Feature Set

In [ ]:
clf = LinearSVC(random_state=SEED)
cv, _ = model_utils.get_cv_iterator(splits)
best_features = model_utils.rfecv_feature_selection(
    clf, X, y, cv, scoring='f1', step=10, verbose=0
)
best_features

## Hyperparameter Tuning

In [ ]:
pipe_clf = Pipeline([
    ('scaler',  MinMaxScaler()),
    ('classifier', clf)
])
param_grid = {'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100, 1000]}
cv = GridSearchCV(
    estimator=pipe_clf, 
    param_grid=param_grid,
    cv=cv,
    verbose=1, 
    scoring='f1',
    n_jobs=-1
)
cv.fit(X[best_features], y)
best_estimator = cv.best_estimator_
print(best_estimator)

## Train Model

In [ ]:
X = data[best_features].fillna(0)
y = data[label]
best_estimator.fit(X, y);

## Prediction: Sliding Window Approach

In [ ]:
areas = [
    'soacha', 
    'bogota', 
    'villadelrosario',
    'uribia', 
    'riohacha', 
    'maicao', 
    'cucuta', 
    'arauca', 
    'arauquita', 
    'tibu'
]

area_dict = geoutils.get_filepaths(areas, images_dir, indices_dir, pos_mask_dir, neg_mask_dir)
grid_blocks = 9

for area in areas:
    output = output_dir + '{}_{}.tif'.format(version, area)
    geoutils.get_preds_windowing(
        area=area, 
        area_dict=area_dict,
        model=best_estimator, 
        tmp_dir=tmp_dir,
        best_features=best_features,  
        output=output, 
        grid_blocks=grid_blocks,
        threshold=0
    )

In [ ]:
#for area in areas:
#    filename = output_dir + '{}_{}.tiff'.format(version, area)
#    !gsutil -q cp {filename} gs://immap-output/